### Initial and boundary value problems of Differential equations
* Ordinary differential equation: one independent variable
 * ODE + initial condition is an initial value problem

* Common tools: **Runge-Kutta**

ODE: $\frac{dy}{dt} = f(t,y)\; $
IC: $y_0 = y(0)$

**Euler method (RK1):**
* Most basic method to solve
* Stepping method to compute next value 

1. Compute approximate derivative with simple forward difference
    $$\frac{dy}{dt} = f(t,y)\; \rightarrow \frac{y_{n+1} - y_n}{h}\approx f(t_n, y_n)$$
2. Use $y_n$ and $t_n$ to compute $y_{n+1}$
    $$ y_{n+1} = y_n + h f(t_n, y_n)$$
3. Continue until end of t

** Euler-Cauchy(RK2):**

1. Compute derivative at initial time (left side of interval)
$$rate_{left} = f(t, y(t)) = f_0$$
2. Use derivative to calculate estimate of derivative at middle of interval
$$y_{mid} = y(t) + \frac{h}{2} rate_{left}$$
3. use mid interval value to estimate rate of change over interval
$$rate_{mid} = f(t+\frac{h}{2}, y_{mid}) = f_1$$
4. Put it all together to calcuate $y_{n+1}$
$$y_{RK2}(t+h) = y(t)+h f\big(t+\frac{h}{2}, y(t)+\frac{h}{2} \; f(t,y(t)) \big)$$


In [ ]:
**Runge-Kutta(RK4):**

